In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
import torch
from nn import MotifCaller, NaiveCaller
from training_data import data_preproc, load_training_data
from utils import get_savepaths
import numpy as np
from sklearn.preprocessing import normalize
from greedy_decoder import GreedyCTCDecoder
from Levenshtein import ratio
from utils import load_model, get_metrics_for_evaluation, sort_transcript
from sklearn.model_selection import train_test_split
from beam_search_decoder import beam_search_ctc
import torch.nn as nn
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [20]:
n_classes = 19
model_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\models\empirical\filtered_model.pth"
model_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\models\empirical\forward.pth"
labels_int = np.arange(n_classes).tolist()
labels = [f"{i}" for i in labels_int] # Tokens to be fed into greedy decoder
greedy_decoder = GreedyCTCDecoder(labels = labels)
ctc = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)

In [21]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_device(device)
model = load_model(model_path=model_path, device=device, n_classes=n_classes)

dataset_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\empirical\pickled_datasets\28_2_25.pkl"
test_dataset_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\datasets\empirical\full_empirical_test_dataset_v5_payload_seq.pkl"
#dataset = pd.read_pickle(dataset_path)
#dataset = pd.read_pickle(test_dataset_path)

#X, y, payloads = load_training_data(
#        test_dataset_path, column_x='squiggle', column_y='motif_seq', payload=True, sampling_rate=0.2)

In [10]:
X, y, payloads = load_training_data(
        test_dataset_path, column_x='squiggle', column_y='Spacer_Sequence', payload=True, sampling_rate=0.2)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

In [12]:
X_train, X_test, payloads_train, payloads_test = train_test_split(
        X, payloads, test_size=0.2, random_state=42)

In [13]:
def evaluate_prediction(prediction, original):

    found = 0
    err = 0
    for i, j in zip(prediction, original):
        for k in range(len(i)):
            if i[k] in j:
                found += 1
            else:
                err += 1

    return found, err

In [24]:

counter = 0
sum_diff = 0

for x, y, payload in tqdm(zip(X_test, y_test, payloads_test), total=len(X_test)):
    input_sequence = torch.tensor(
        x, dtype=torch.float32)
    input_sequence = input_sequence.view(
        1, 1, len(x)).to(device)
    model_output = model(input_sequence)
    model_output = model_output.permute(1, 0, 2)
    
    label_lengths = torch.tensor([len(y)])
    target_sequence = torch.tensor(y).to(device)

    
    n_timesteps = model_output.shape[0]
    input_lengths = torch.tensor([n_timesteps])
    
    model_output_flattened = model_output.view(
        model_output.shape[0] * model_output.shape[1], n_classes)

    loss = ctc(
        model_output, target_sequence, input_lengths, label_lengths)
    print(loss.item())
    
    greedy_transcript = " ".join(greedy_decoder(model_output))
    beam_transcript = beam_search_ctc(
        model_output_flattened.detach().cpu(), beam_width=20)
    actual_transcript = " ".join([str(i) for i in y])

    decoded_prediction = sort_transcript(greedy_transcript)
    search_prediction = sort_transcript(actual_transcript)
    original = sort_transcript(" ".join([str(i) for i in payload]))
    #print(decoded_prediction)
    #print(search_prediction)
    #print(original)
    
    found_motifs_caller = evaluate_prediction(decoded_prediction, original)
    found_motifs_search = evaluate_prediction(search_prediction, original)
    print(found_motifs_caller)
    print(found_motifs_search)
    sum_diff += found_motifs_caller[0] - found_motifs_search[0]
    
    #greedy_ratio = ratio(greedy_transcript, actual_transcript)
    #beam_ratio = ratio(beam_transcript, actual_transcript)
    #sum_diff += beam_ratio - greedy_ratio
    #print()
    counter+=1
    print(sum_diff)
    print()

    if counter == 200:
        print(sum_diff)
        break

  0%|          | 0/2762 [00:00<?, ?it/s]

2.9585866928100586
(3, 1)
(4, 0)
-1

17.379819869995117
(2, 3)
(1, 0)
0

15.683625221252441
(0, 4)
(1, 0)
-1

7.839217185974121
(2, 2)
(2, 0)
-1

6.514053821563721
(4, 1)
(2, 0)
1

8.028848648071289
(4, 0)
(5, 0)
0

8.212181091308594
(2, 2)
(2, 0)
0

16.8619441986084
(1, 4)
(1, 0)
0

16.176551818847656
(2, 2)
(1, 0)
1

2.840304136276245
(2, 2)
(3, 0)
0

16.70201301574707
(1, 4)
(1, 0)
0

12.562923431396484
(2, 2)
(1, 0)
1

10.590460777282715
(1, 4)
(3, 0)
-1

6.689669132232666
(2, 2)
(5, 0)
-4

15.859752655029297
(5, 0)
(1, 0)
0

4.709817409515381
(2, 3)
(4, 0)
-2

13.006366729736328
(1, 4)
(1, 0)
-2

8.198397636413574
(3, 2)
(3, 0)
-2

18.26764488220215
(2, 3)
(1, 0)
-1

18.063770294189453
(0, 5)
(1, 0)
-2

7.439448356628418
(4, 0)
(5, 2)
-3

7.183128356933594
(2, 2)
(2, 0)
-3

4.571244239807129
(1, 3)
(4, 0)
-6

14.714981079101562
(3, 2)
(1, 0)
-4

13.256802558898926
(2, 3)
(1, 0)
-3

5.335145950317383
(3, 2)
(5, 0)
-5

2.731339454650879
(4, 1)
(4, 0)
-5

6.005348205566406
(1, 3)
(2,

KeyboardInterrupt: 

In [26]:
1 * 100 /32

3.125

In [38]:
sum_diff

71